In [ ]:
import os
from os import cpu_count
from math import floor
import pandas as pd
import numpy as np
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [ ]:
data=pd.read_parquet("/kaggle/input/nfcsecicids2018v2/NF-CSE-CIC-IDS2018-V2.parquet")

## very very Basic EDA

In [ ]:
data.dtypes

In [ ]:
data.Label.value_counts()

In [ ]:
data.Attack.value_counts()

In [ ]:
ddos=data[data['Attack'].str.contains("ddos",case=False)]
dos=data[data['Attack'].str.contains("dos",case=False)]
data.drop(index=dos.index,inplace=True)
dos.drop(index=ddos.index,inplace=True)

brute=data[data['Attack'].str.contains("brute",case=False)]
data.drop(index=brute.index,inplace=True)

print("before subsampling")
print(dos.Attack.value_counts())
print(ddos.Attack.value_counts())
print(brute.Attack.value_counts())

grouped = dos.groupby(dos.Attack)
dos_attacks=[ grouped.get_group(attack).sample(9512) for attack in dos.Attack.unique() ]
dos=pd.concat(objs=dos_attacks)

grouped = ddos.groupby(ddos.Attack)
ddos_attacks=[ grouped.get_group(attack).sample(2112) for attack in ddos.Attack.unique() ]
ddos=pd.concat(objs=ddos_attacks)

grouped = brute.groupby(brute.Attack)
brute_attacks=[ grouped.get_group(attack).sample(927) for attack in brute.Attack.unique() ]
brute=pd.concat(objs=brute_attacks)


print("after subsampling")
print(dos.Attack.value_counts())
print(ddos.Attack.value_counts())
print(brute.Attack.value_counts())

dos.Attack="DoS"
ddos.Attack="DDoS"
brute.Attack="Brute Force"
data=pd.concat(objs=[data, dos,ddos,brute])

In [ ]:
data=data.drop(columns=['L4_SRC_PORT', 'L4_DST_PORT']) #dropping metadata
data=data[data["Attack"]!="Infilteration"]
# drop infiltration attacks because of inherently low classification score, results become less reliable when used

## Finding contaminant features using XAI (SHAP)
goal: 
- try to find the contaminant features that have blanked predictive power across all attack classes

method:
* look for shap values per feature that are consistantly high or low among all attack classes.
   
how:
```
possible_contaminants =[]
    while models can achive high accuracy{
    
        * Train a binary classification model (e.g., xgboost) on different attack classes and benign class
        * For each model, calculate shappley values on test set
        * For each model, use np.abs(shap_values).mean(0) to compute the mean absolute SHAP value for each feature across all samples. 
          This will give you a measure of how much each feature contributes to the model output on average across all samples (the importance).
          
        * normalize importances across for each attack class sum of feature importances equals 1
        * for each feature, calculate variance of importance for each attack class
        * for each feature, calculate average importance across all attack classes
        * score of a feature is the weighted sum of the importance and variance
        
        * feature F is feature with the highest score
        * possible_contaminants.append(F)
        * drop F from dataset
    }
```

In [ ]:
training_set = data.sample(frac=0.3, replace=False,random_state=42)
testing_set = data.drop(index=training_set.index)

In [ ]:
training_set.Attack.value_counts()

In [ ]:
attacks=training_set.Attack.unique()
attacks

### split into dataframes per attack class

In [ ]:
grouped = training_set.groupby(training_set.Attack)
dfs={cat:grouped.get_group(cat) for cat in attacks[1:]}
dfs[attacks[0]]=grouped.get_group(attacks[0]) #don't subsample normal attacks
#dfs key=attack_cat, value is dataframe 

In [ ]:
for atk_type in attacks[1:]:
    normals_to_sample = dfs[atk_type].shape[0]
#     print(normals_to_sample)
    normals_sample = dfs['Benign'].sample(normals_to_sample)
#     dfs['Normal'] = dfs['Normal'].drop(index=normals_sample.index)#don't resample
    dfs[atk_type] = pd.concat(objs=[dfs[atk_type], normals_sample])
    print(dfs[atk_type].shape[0], normals_sample.shape[0], dfs['Benign'].shape[0])

In [ ]:
testing_dfs = {}
grouped = testing_set.groupby(testing_set.Attack)
testing_dfs={cat:grouped.get_group(cat) for cat in attacks[1:]}
testing_dfs[attacks[0]]=grouped.get_group(attacks[0]) #don't subsample normal attacks
#testing_dfs key=attack_cat, value is dataframe 

In [ ]:
for atk_type in attacks[1:]:
    normals_to_sample = testing_dfs[atk_type].shape[0] 
    normals_sample = testing_dfs['Benign'].sample(normals_to_sample)
#     testing_dfs['Normal'] = testing_dfs['Normal'].drop(index=normals_sample.index)#don't resample
    testing_dfs[atk_type] = pd.concat(objs=[testing_dfs[atk_type], normals_sample])
    print(testing_dfs[atk_type].shape[0], normals_sample.shape[0], testing_dfs['Benign'].shape[0])

In [ ]:
print(f"TRAINING SETS")
for k,v in dfs.items():
    v.drop(columns=['Attack'], inplace=True)   
    print(k, v.shape)
    print(v['Label'].value_counts())

In [ ]:
print("TESTING SETS")
for k,v in testing_dfs.items():
    v.drop(columns=['Attack'], inplace=True)   
    print(k, v.shape)
    print(v['Label'].value_counts())

In [ ]:
training_dfs=dfs

In [ ]:
import plotly.express as px

### L7_PROTO

In [ ]:
import math
for attack in attacks[1:]:
    d=training_dfs[attack]#[training_dfs[attack].ICMP_TYPE<=1000000]
    k=d#[training_dfs[attack].Label==1]
#     k=k[k.DST_TO_SRC_AVG_THROUGHPUT>0]
    bin_width= 1000
    # here you can choose your rounding method, I've chosen math.ceil
    nbins = math.ceil((k["L7_PROTO"].max() - k["L7_PROTO"].min()) / bin_width)
    fig=px.histogram(
    k,
    x='L7_PROTO',
    color='Label',
    marginal='box',
    barmode='group',#nbins=nbins,
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
training_set.L7_PROTO

In [ ]:
k=training_set.L7_PROTO.astype("int64").value_counts()
print(k[k>10])

In [ ]:
training_set.groupby('Attack')['L7_PROTO'].apply(lambda x: ((x//1==0) | (x//1==5) | (x//1==91)).mean() * 100)

### min_ttl

In [ ]:
px.histogram(training_set,x='MIN_TTL',color='Label',barmode='group',marginal='box',histnorm='percent')

In [ ]:
training_set.groupby('Attack')['MIN_TTL'].apply(lambda x: ((x==63)|(x==128)|(x==127)).mean() * 100)

In [ ]:
training_set.groupby(["Label","MIN_TTL"]).MIN_TTL.count()

In [ ]:
training_set[training_set.Attack=='DoS'].MIN_TTL

### TCP_WIN_MAX_IN

In [ ]:
for attack in attacks[1:]:
    fig=px.histogram(
    training_dfs[attack],
    x='TCP_WIN_MAX_IN',
    color='Label',
    marginal='box',
    barmode='group',
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
training_set.groupby('Attack')['TCP_WIN_MAX_IN'].apply(lambda x: (x ==0).mean() * 100)

### TCP_WIN_MAX_OUT

In [ ]:
for attack in attacks[1:]:
    fig=px.histogram(
    training_dfs[attack],
    x='TCP_WIN_MAX_OUT',
    color='Label',
    marginal='box',
    barmode='group',
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
training_set.groupby('Attack')['TCP_WIN_MAX_OUT'].apply(lambda x: ((x==0)|(x==26883)|(x==26847)).mean() * 100)

### FLOW_DURATION_MILLISECONDS

In [ ]:
for attack in attacks[1:]:
    fig=px.histogram(
    training_dfs[attack],
    x='FLOW_DURATION_MILLISECONDS',
    color='Label',
    marginal='box',
    barmode='group',
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
training_set.groupby('Attack')['FLOW_DURATION_MILLISECONDS'].apply(lambda x: (x >4000000).mean() * 100)

### MIN_TTL

In [ ]:
# import plotly.express as px
px.histogram(training_set[training_set.Label==1],x='MIN_TTL',color='Label',barmode='group',marginal="box")

### IN_BYTES

In [ ]:
import math
for attack in attacks[1:]:
    d=training_dfs[attack]#[training_dfs[attack].SRC_TO_DST_SECOND_BYTES<=1000000]
    k=d#[training_dfs[attack].Label==1]
#     k=k[k.DST_TO_SRC_AVG_THROUGHPUT>0]
    bin_width= 500
    # here you can choose your rounding method, I've chosen math.ceil
    nbins = math.ceil((k["IN_BYTES"].max() - k["IN_BYTES"].min()) / bin_width)
    fig=px.histogram(
    k,
    x='IN_BYTES',
    color='Label',
    marginal='box',
    barmode='group',nbins=nbins,
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
(training_set.groupby('Attack')['IN_BYTES'].apply(lambda x: (x>=500).mean())*100).reset_index(name='Percentage')

### TCP_WIN_MAX_IN

In [ ]:
for attack in attacks[1:]:
    fig=px.histogram(
    training_dfs[attack],
    x='TCP_WIN_MAX_IN',
    color='Label',
    marginal='box',
    barmode='group',
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
k=training_set.groupby(["Label","TCP_WIN_MAX_IN"]).TCP_WIN_MAX_IN.count()

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(k[k>30])

In [ ]:
training_set.groupby('Attack')['TCP_WIN_MAX_IN'].apply(lambda x: ((x==26883)).mean() * 100)

In [ ]:
training_set.groupby('Attack')['TCP_WIN_MAX_IN'].apply(lambda x: ((x==0)|(x==8192)|(x==26883)|(x==65535)).mean() * 100)

### DURATION_OUT

In [ ]:
for attack in attacks[1:]:
    fig=px.histogram(
    training_dfs[attack],#[training_dfs[attack].Label>0],
    x='DURATION_OUT',
    color='Label',
    marginal='box',
    barmode='group',
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
training_set.groupby('Attack')['DURATION_OUT'].apply(lambda x: ((x>0)).mean() * 100)

### NUM_PKTS_128_TO_256_BYTES

In [ ]:
for attack in attacks[1:]:
    fig=px.histogram(
    training_dfs[attack],#[training_dfs[attack].Label>0],
    x='NUM_PKTS_128_TO_256_BYTES',
    color='Label',
    marginal='box',
    barmode='group',
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
(training_set.groupby('Attack')['NUM_PKTS_128_TO_256_BYTES'].apply(lambda x: (x==0).mean())*100).reset_index(name='Percentage')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(3, 2, figsize=(12, 9))
axes = axes.flatten()

colors = ["#00FF00","#FF0000"]


for i, attack in enumerate(attacks[1:]):
    sns.histplot(training_dfs[attack],
                 x='TCP_WIN_MAX_IN',
                 hue='Label',
                 stat='percent',palette=colors,
                 ax=axes[i])
    axes[i].set_title(attack)
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Create subplots with 1 row and 2 columns
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

# Plot histogram for "Fuzzers" attack class
sns.histplot(dfs["Bot"][dfs["Bot"].Label==1].MIN_TTL, stat="percent", ax=axs[0])
axs[0].set_title("Fuzzers Attack Class")

# Plot histogram for "Backdoor" attack class
sns.histplot(dfs["DoS"][dfs["DoS"].Label==1].MIN_TTL, stat="percent", ax=axs[1])
axs[1].set_title("Backdoor Attack Class")

# Set common y-axis label
fig.text(0.5, 0.04, "MIN_TTL", ha='center')

# Adjust spacing between subplots
plt.tight_layout()

# Show the plot
plt.show()

### TCP_FLAGS

In [ ]:
for attack in attacks[1:]:
    fig=px.histogram(
    training_dfs[attack],
    x='TCP_FLAGS',
    color='Label',
    marginal='box',
    barmode='group',
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
(training_set.groupby('Attack')['TCP_FLAGS'].apply(lambda x: ((x==in)).mean())*100).reset_index(name='Percentage')

### LONGEST_FLOW_PKT

In [ ]:
import math
for attack in attacks[1:]:
#     d=training_dfs[attack][training_dfs[attack].DST_TO_SRC_SECOND_BYTES<=1000000]
    k=training_dfs[attack]#[training_dfs[attack].Label==1]
#     k=k[k.DST_TO_SRC_AVG_THROUGHPUT>0]
    bin_width= 10
    # here you can choose your rounding method, I've chosen math.ceil
    nbins = math.ceil((k["LONGEST_FLOW_PKT"].max() - k["LONGEST_FLOW_PKT"].min()) / bin_width)
    fig=px.histogram(
    k,
    x='LONGEST_FLOW_PKT',
    color='Label',
    marginal='box',
    barmode='group',#nbins=nbins,
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
k=training_set[training_set.Label==1].LONGEST_FLOW_PKT.value_counts()

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(k[k>10])

In [ ]:
(training_set.groupby('Attack')['LONGEST_FLOW_PKT'].apply(lambda x: ((x==60)|(x==987)|(x==1024)|(x==366)|(x==975)).mean())*100).reset_index(name='Percentage')

### SRC_TO_DST_AVG_THROUGHPUT

In [ ]:
import math
for attack in attacks[1:]:
    d=training_dfs[attack][training_dfs[attack].SRC_TO_DST_AVG_THROUGHPUT<=1000000]
    k=d#[d.Label==1]
#     k=k[k.DST_TO_SRC_AVG_THROUGHPUT>0]
    bin_width= 100000
    # here you can choose your rounding method, I've chosen math.ceil
    nbins = math.ceil((k["SRC_TO_DST_AVG_THROUGHPUT"].max() - k["SRC_TO_DST_AVG_THROUGHPUT"].min()) / bin_width)
    fig=px.histogram(
    k,
    x='SRC_TO_DST_AVG_THROUGHPUT',
    color='Label',
    marginal='box',
    barmode='group',#nbins=nbins,
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
(training_set.groupby('Attack')['SRC_TO_DST_AVG_THROUGHPUT'].apply(lambda x: ((x<400000)).mean())*100).reset_index(name='Percentage')

### OUT_BYTES

In [ ]:
import math
for attack in attacks[1:]:
    d=training_dfs[attack][training_dfs[attack].DST_TO_SRC_SECOND_BYTES<=1000000]
    k=d#[d.Label==1]
    bin_width= 100
    # here you can choose your rounding method, I've chosen math.ceil
    nbins = math.ceil((k["OUT_BYTES"].max() - k["OUT_BYTES"].min()) / bin_width)
    fig=px.histogram(
    k,
    x='OUT_BYTES',
    color='Label',
    marginal='box',
    barmode='group',#nbins=nbins,
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
(training_set.groupby('Attack')['OUT_BYTES'].apply(lambda x: ((x>175)).mean())*100).reset_index(name='Percentage')

In [ ]:
# k=training_set.groupby(["Label","OUT_BYTES"]).OUT_BYTES.value_counts()

In [ ]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     print(k[k>1000])

### SRC_TO_DST_SECOND_BYTES

In [ ]:
import math
for attack in attacks[1:]:
    d=training_dfs[attack][training_dfs[attack].SRC_TO_DST_SECOND_BYTES<=1000000]
    k=d#[training_dfs[attack].Label==1]
#     k=k[k.DST_TO_SRC_AVG_THROUGHPUT>0]
    bin_width= 100
    # here you can choose your rounding method, I've chosen math.ceil
    nbins = math.ceil((k["SRC_TO_DST_SECOND_BYTES"].max() - k["SRC_TO_DST_SECOND_BYTES"].min()) / bin_width)
    fig=px.histogram(
    k,
    x='SRC_TO_DST_SECOND_BYTES',
    color='Label',
    marginal='box',
    barmode='group',#nbins=nbins,
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
(training_set.groupby('Attack')['SRC_TO_DST_SECOND_BYTES'].apply(lambda x: ((x<=780)).mean())*100).reset_index(name='Percentage')

### DST_TO_SRC_SEOND+BYTES HIGH CORRELATION WITH ABOVE

In [ ]:
import math
for attack in attacks[1:]:
    d=training_dfs[attack][training_dfs[attack].DST_TO_SRC_SECOND_BYTES<=1000000]
    k=d#[d.Label==1]
#     k=k[k.DST_TO_SRC_AVG_THROUGHPUT>0]
    bin_width= 100
    # here you can choose your rounding method, I've chosen math.ceil
    nbins = math.ceil((k["DST_TO_SRC_SECOND_BYTES"].max() - k["DST_TO_SRC_SECOND_BYTES"].min()) / bin_width)
    fig=px.histogram(
    k,
    x='DST_TO_SRC_SECOND_BYTES',
    color='Label',
    marginal='box',
    barmode='group',#nbins=nbins,
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
(training_set.groupby('Attack')['DST_TO_SRC_SECOND_BYTES'].apply(lambda x: ((x>300)).mean())*100).reset_index(name='Percentage')

### DST_TO_SRC_AVG_THROUGHPUT

In [ ]:
import math
for attack in attacks[1:]:
#     d=training_dfs[attack][training_dfs[attack].DST_TO_SRC_SECOND_BYTES<=1000000]
    k=training_dfs[attack]#[training_dfs[attack].Label==1]
    k=k#[k.DST_TO_SRC_AVG_THROUGHPUT>0]
    bin_width= 100000
    # here you can choose your rounding method, I've chosen math.ceil
    nbins = math.ceil((k["DST_TO_SRC_AVG_THROUGHPUT"].max() - k["DST_TO_SRC_AVG_THROUGHPUT"].min()) / bin_width)
    fig=px.histogram(
    k,
    x='DST_TO_SRC_AVG_THROUGHPUT',
    color='Label',
    marginal='box',
    barmode='group',#nbins=nbins,
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
(training_set.groupby('Attack')['DST_TO_SRC_AVG_THROUGHPUT'].apply(lambda x: ((x<1000000)).mean())*100).reset_index(name='Percentage')

In [ ]:
(training_set.groupby('Attack')['DST_TO_SRC_AVG_THROUGHPUT'].apply(lambda x: ((x<1496000)).mean())*100).reset_index(name='Percentage')

### SERVER_TCP_FLAGS

In [ ]:
import math
for attack in attacks[1:]:
    d=training_dfs[attack]#[training_dfs[attack].SRC_TO_DST_SECOND_BYTES<=1000000]
    k=d#[training_dfs[attack].Label==1]
#     k=k[k.DST_TO_SRC_AVG_THROUGHPUT>0]
    bin_width= 100
    # here you can choose your rounding method, I've chosen math.ceil
    nbins = math.ceil((k["SERVER_TCP_FLAGS"].max() - k["SERVER_TCP_FLAGS"].min()) / bin_width)
    fig=px.histogram(
    k,
    x='SERVER_TCP_FLAGS',
    color='Label',
    marginal='box',
    barmode='group',#nbins=nbins,
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
np.corrcoef([training_set.SERVER_TCP_FLAGS,training_set.CLIENT_TCP_FLAGS,training_set.TCP_FLAGS])

In [ ]:
(training_set.groupby('Attack')['SERVER_TCP_FLAGS'].apply(lambda x: ((x==27)).mean())*100).reset_index(name='Percentage')

### NUM_PKTS_512_TO_1024_BYTES

In [ ]:
import math
for attack in attacks[1:]:
    d=training_dfs[attack]#[training_dfs[attack].NUM_PKTS_512_TO_1024_BYTES<=10000]
    k=d#[training_dfs[attack].Label==1]
#     k=k[k.DST_TO_SRC_AVG_THROUGHPUT>0]
    bin_width= 5
    # here you can choose your rounding method, I've chosen math.ceil
    nbins = math.ceil((k["NUM_PKTS_512_TO_1024_BYTES"].max() - k["NUM_PKTS_512_TO_1024_BYTES"].min()) / bin_width)
    fig=px.histogram(
    k,
    x='NUM_PKTS_512_TO_1024_BYTES',
    color='Label',
    marginal='box',
    barmode='group',#nbins=nbins,
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
(training_set.groupby('Attack')['NUM_PKTS_512_TO_1024_BYTES'].apply(lambda x: ((x<=1)).mean())*100).reset_index(name='Percentage')


In [ ]:
import math
for attack in attacks[1:]:
    d=training_dfs[attack]#[training_dfs[attack].NUM_PKTS_512_TO_1024_BYTES<=10000]
    k=d[training_dfs[attack].Label==1]
#     k=k[k.DST_TO_SRC_AVG_THROUGHPUT>0]
    bin_width= 5
    # here you can choose your rounding method, I've chosen math.ceil
    nbins = math.ceil((k["NUM_PKTS_512_TO_1024_BYTES"].max() - k["NUM_PKTS_512_TO_1024_BYTES"].min()) / bin_width)
    fig=px.histogram(
    k,
    x='NUM_PKTS_256_TO_512_BYTES',
    color='Label',
    marginal='box',
    barmode='group',#nbins=nbins,
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
(training_set.groupby('Attack')['NUM_PKTS_256_TO_512_BYTES'].apply(lambda x: ((x<=1)).mean())*100).reset_index(name='Percentage')

### SHORTEST_FLOW_PKT

In [ ]:
for attack in attacks[1:]:
    fig=px.histogram(
    training_dfs[attack],
    x='SHORTEST_FLOW_PKT',
    color='Label',
    marginal='box',
    barmode='group',
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
(training_set.groupby('Attack')['SHORTEST_FLOW_PKT'].apply(lambda x: ((x==40)|(x==52)|(x==60)).mean())*100).reset_index(name='Percentage')

### MIN_IP_PKT_LEN

In [ ]:
for attack in attacks[1:]:
    fig=px.histogram(
    training_dfs[attack],
    x='MIN_IP_PKT_LEN',
    color='Label',    barmode='group',

    marginal='box',
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
(training_set.groupby('Attack')['MIN_IP_PKT_LEN'].apply(lambda x: ((x==52)|(x==40)|(x==0)).mean())*100).reset_index(name='Percentage')

### ICMP-TYPE

In [ ]:
import math
for attack in attacks[1:]:
    d=training_dfs[attack]#[training_dfs[attack].ICMP_TYPE<=1000000]
    k=d[training_dfs[attack].Label==1]
#     k=k[k.ICMP_TYPE>0]
    bin_width= 1000
    # here you can choose your rounding method, I've chosen math.ceil
#     nbins = math.ceil((k["ICMP_TYPE"].max() - k["ICMP_TYPE"].min()) / bin_width)
    fig=px.histogram(
    k,
    x='ICMP_TYPE',
    color='Label',
    marginal='box',
    barmode='group',#nbins=nbins,
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
(training_set.groupby('Attack')['ICMP_TYPE'].apply(lambda x: ((x==0)|(x==0)|(x==0)).mean())*100).reset_index(name='Percentage')

### PROTOCOL

In [ ]:
import math
for attack in attacks[1:]:
    d=training_dfs[attack]#[training_dfs[attack].ICMP_TYPE<=1000000]
    k=d#[training_dfs[attack].Label==1]
#     k=k[k.DST_TO_SRC_AVG_THROUGHPUT>0]
    bin_width= 1000
    # here you can choose your rounding method, I've chosen math.ceil
    nbins = math.ceil((k["PROTOCOL"].max() - k["PROTOCOL"].min()) / bin_width)
    fig=px.histogram(
    k,
    x='PROTOCOL',
    color='Label',
    marginal='box',
    barmode='group',#nbins=nbins,
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
training_set.groupby('Attack')['PROTOCOL'].apply(lambda x : ((x==6)|(x==17)).mean()*100)

### NUM_PKTS_UP_TO_128_BYTES

In [ ]:
import math 
for attack in attacks[1:]:
#     d=training_dfs[attack][training_dfs[attack].DST_TO_SRC_SECOND_BYTES<=1000000]
    k=training_dfs[attack]#[training_dfs[attack].Label==1]
    k=k[k.NUM_PKTS_UP_TO_128_BYTES>0]
    bin_width= 100
#     here you can choose your rounding method, I've chosen math.ceil
    nbins = math.ceil((k["NUM_PKTS_UP_TO_128_BYTES"].max() - k["NUM_PKTS_UP_TO_128_BYTES"].min()) / bin_width)
    fig=px.histogram(
    k,
    x='NUM_PKTS_UP_TO_128_BYTES',
    color='Label',
    marginal='box',
    barmode='group',
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
training_set.groupby('Attack')['NUM_PKTS_UP_TO_128_BYTES'].apply(lambda x: ((x==8)).mean() * 100)

### IN_PKTS

In [ ]:
import math
for attack in attacks[1:]:
#     d=training_dfs[attack][training_dfs[attack].DST_TO_SRC_SECOND_BYTES<=1000000]
    k=training_dfs[attack]#[training_dfs[attack].Label==1]
#     k=k[k.DST_TO_SRC_AVG_THROUGHPUT>0]
    bin_width= 2
    # here you can choose your rounding method, I've chosen math.ceil
    nbins = math.ceil((k["IN_PKTS"].max() - k["IN_PKTS"].min()) / bin_width)
    fig=px.histogram(
    k,
    x='IN_PKTS',
    color='Label',
    marginal='box',
    barmode='group',#nbins=nbins,
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
(training_set.groupby('Attack')['IN_PKTS'].apply(lambda x: ((x.between(0,4)) | (x.between(10,14))).mean())*100).reset_index(name='Percentage')

In [ ]:
(training_set.groupby('Attack')['IN_PKTS'].apply(lambda x: (x>4).mean())*100).reset_index(name='Percentage')

### CLIENT_TCP_FLAGS

In [ ]:
for attack in attacks[1:]:
    fig=px.histogram(
    training_dfs[attack],
    x='CLIENT_TCP_FLAGS',
    color='Label',
    marginal='box',
    barmode='group',
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()

In [ ]:
training_set.groupby('Attack')['CLIENT_TCP_FLAGS'].apply(lambda x: ((x.between(27,31))|(x.between(219,222))|(x ==30)).mean() * 100)

In [ ]:
training_set.groupby('Attack')['CLIENT_TCP_FLAGS'].apply(lambda x: ((x ==219)|(x ==0)|(x ==219)).mean() * 100)

### FLOW_DURATION_MILLISECONDS

In [ ]:
for attack in attacks[1:]:
    fig=px.histogram(
    training_dfs[attack],
    x='FLOW_DURATION_MILLISECONDS',
    color='Label',
    marginal='box',
    barmode='group',
    color_discrete_sequence=['turquoise','blue'],histnorm='percent',title=attack)
    fig.show()